# RAG with an Agent
* We will build a basic agent for doing retrieval when necessary and also holding a conversation.

## Dependencies
* Note: we will use a FAISS vector database.

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## .env File
* Our LangSmith project name: RAGwithAgent

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Open LangSmith to track the following operations
* smith.langchain.com

## First we will build the retriever

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/state_of_the_union.txt")
documents = loader.load()

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [5]:
retriever = db.as_retriever()

## Then we will create a tool so the agent can use the previous retriever

In [6]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns excerpts from the 2022 State of the Union.",
)
tools = [tool]

## Now we will build the Agent

In [7]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [9]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

## We can now have a normal conversation with the Agent

In [10]:
result = agent_executor.invoke({"input": "hi, im bob"})

In [11]:
result["output"]

'Hello Bob! How can I assist you today?'

## The Agent will do RAG when we ask someting related with the private document

In [12]:
result = agent_executor.invoke(
    {
        "input": "what did the president say about ketanji brown jackson in the most recent state of the union?"
    }
)

In [13]:
result["output"]

"In the most recent State of the Union address, President Biden honored Justice Stephen Breyer and announced the nomination of Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court. President Biden described Judge Ketanji Brown Jackson as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. The President highlighted her qualifications and the broad range of support she has received since her nomination."

In [14]:
result = agent_executor.invoke(
    {"input": "how long ago did the president nominate ketanji brown jackson?"}
)

In [15]:
result["output"]

'President Biden nominated Ketanji Brown Jackson 4 days ago.'